In [1]:
import cv2
import cvzone
import numpy as np
from cvzone.HandTrackingModule import HandDetector

In [17]:
allowedLenght = 150
allowedLenght_rate= 50
error_rate = 0.1

In [18]:
class SnakeGameClass:
    def __init__(self, pathFood):
        self.points = [] #list of all points on the snake
        self.lengths = [] #ditances between all points of the snake
        self.score = 0
        self.previous_score = 0
        self.GameOver = False
        self.currentLength = 0
        self.allowedLength = allowedLenght
        self.previousHead = 0, 0
        self.imgFood = cv2.imread(pathFood, cv2.IMREAD_UNCHANGED)
        self.hFood, self.wFood , _ = self.imgFood.shape
        self.foodPoint = 0, 0
        self.randomfoodLocation()
    
    def randomfoodLocation(self):
        self.foodPoint = np.random.randint(100, 1000), np.random.randint(100, 600)
    
    def update(self, imgMain, currentHead):
        if self.GameOver:
            cvzone.putTextRect(imgMain, 'Game Over', [300, 400], scale=7, thickness=5, offset=20)
            cvzone.putTextRect(imgMain, f'Score: {self.previous_score}', [300, 500], scale=7, thickness=5, offset=20)            
        else:
            px, py = self.previousHead
            cx, cy = currentHead
            
            self.points.append([cx, cy])
            distance = ((cx - px)**2 + (cy - py)**2)**0.5
            self.lengths.append(distance)
            self.currentLength += distance
            self.previousHead = cx, cy
            
            #length reduction
            if len(self.points) != 0 and (self.currentLength > self.allowedLength):
                for i, length in enumerate(self.lengths):
                    self.currentLength -= length
                    self.lengths.pop(i)
                    self.points.pop(i)
                    
                    if self.currentLength < self.allowedLength:
                        break
            
            #check if the snake ate the food
            rx, ry = self.foodPoint
            if rx - self.wFood//2 < cx < rx + self.wFood//2 and ry - self.hFood//2 < cy < ry + self.hFood//2:
                self.randomfoodLocation()
                self.allowedLength += allowedLenght_rate
                self.score += 1
                print(f'Current Score: {self.score}')
            
            #draw the snake
            for i in range(1, len(self.points)):
                cv2.line(imgMain, self.points[i-1], self.points[i], (32, 165, 218), 20)
                
            cv2.circle(imgMain, pointIndex, 20, (100, 0, 0), cv2.FILLED)
            
            #check for collision
            pts = np.array(self.points[:-6], np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(imgMain, [pts], False, (0, 0, 255), 3)
            mindistance = cv2.pointPolygonTest(pts, (cx, cy), True)
            
            if -error_rate < mindistance < error_rate:
                print('Hit')
                self.points = [] #list of all points on the snake
                self.lengths = [] #ditances between all points of the snake
                self.previous_score = self.score
                self.score = 0
                self.GameOver = True
                self.currentLength = 0
                self.allowedLength = allowedLenght
                self.previousHead = 0, 0
                self.randomfoodLocation()
            
            #Draw Food
            imgMain = cvzone.overlayPNG(imgMain, self.imgFood, (rx-self.wFood//2, ry-self.hFood//2))
            cvzone.putTextRect(imgMain, f'Score: {self.score}', [50, 80], scale=3, thickness=3, offset=10)
        return imgMain

In [ ]:
game = SnakeGameClass('Donut.png')

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

detector = HandDetector(detectionCon=0.8, maxHands=1)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1) #Rotate image, the image won't effect like a mirror
    hands, img = detector.findHands(img, flipType=False)
    
    if hands:
        lmList = hands[0]['lmList']
        pointIndex = lmList[8][0:2] #point figure(x, y, z)  ===> (x, y)
        img = game.update(img, pointIndex)
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if cv2.waitKey(1) & 0xFF == ord('r'):
        game.GameOver = False
        print("Restart the Game")
 
cap.release()
cv2.destroyAllWindows()


Current Score: 1
Current Score: 2
Current Score: 3
Current Score: 4
Current Score: 5
Current Score: 6
Current Score: 7
Current Score: 8
Current Score: 9
Current Score: 10
Current Score: 11
Current Score: 12
Current Score: 13
Current Score: 14
Current Score: 15
Current Score: 16
Current Score: 17
Current Score: 18
Current Score: 19
Hit
Restart the Game
Current Score: 1
Current Score: 2
Current Score: 3
Current Score: 4
Current Score: 5
Current Score: 6
Current Score: 7
Hit
Restart the Game
Current Score: 1
Current Score: 2
Current Score: 3
Current Score: 4
Current Score: 5
Current Score: 6
Current Score: 7
Current Score: 8
Current Score: 9
Current Score: 10
Current Score: 11
Current Score: 12
